# Datan analysointi K2020_2 - Tehtävät kerta 2 - RATKAISUT

Palauta Optiman palautuslaatikkoon viimeistään su 2.2.  Kirjoita palautuslaatikon kommenttikenttään, mitä tehtävät olet tehnyt kokonaan/osittain.

**Tee tehtävät ilman for-silmukoita tms.**

**Opiskelijan nimi**:

## 1. Pelaajat

Lue tiedosto https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/nhl1.csv DataFrameen siten, että otat mukaan vain seuraavat sarakkeet ja vaihda sarakkeiden otsikot suluissa oleviksi suomenkielisiksi:  
  * Player      (pelaaja)
  * Team        (joukkue)
  * Birth City  (syntymäkaupunki)
  * Ntnlty      (kansalaisuus)    
  * Ht          (pituus)  
  * Wt          (paino)
  * Overall     (varausnro)  
  * GP          (pelit)
  * G           (maalit)
  
Pituus on tuumina, muunna se senttimetreiksi kertomalla 2,54:llä.  

Paino on nauloina, muunna se kilogrammoiksi kertomalla 0,453592:llä ja pyöristämällä tulos 0,1 kg tarkkuuteen.

Luo uusi sarake jossa on paino-tiedon perusteella "painoluokka" 5 kg välein lokeroituna. (Lue ensin datasta suurin ja pienin paino)

Laske sitten kuhunkin painoluokkaan kuuluvien rivien määrä Serieksen `value_counts()`-metodilla.

Tulosta lopuksi ne pelaajat joiden syntymäkaupunki on Jyväskylä (kaikki sarakkeet).

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/nhl1.csv ', 
                 usecols=['Player', 'Team', 'Birth City', 'Ntnlty', 'Ht', 'Wt', 'GP', 'P' ]) 
                 
df.columns= ['pelaaja','joukkue', 'syntymäkaupunki', 'maa', 'pituus', 'paino','ottelut','pisteet']

# tapa2
#df = pd.read_csv('https://student.labranet.jamk.fi/~varpe/datananal1k2020/kerta2/nhl1.csv', 
#                 usecols=[0, 1, 5, 8, 9, 10, 16, 19],
#                 names = ['pelaaja','joukkue', 'syntymäkaupunki', 'maa', 'pituus', 'paino','ottelut','pisteet'],
#                 skiprows = 1)

# muunnokset, ei tarvita applyä näissä
df['pituus'] = df['pituus']*2.54
df['paino'] = (df['paino']*0.453592).round(1)


# painoluokkalokerot, roundilla laitetaan rajat tasakiloiksi
bins = np.arange(round(df['paino'].min()-0.5,0),df['paino'].max()+5,5)

df['painoluokka'] = pd.cut(df['paino'], bins)

print(df['painoluokka'].value_counts(sort=False))

print('\n--------------\n')

print(df[df['syntymäkaupunki']=='Jyväskylä'])

(69.0, 74.0]        8
(74.0, 79.0]       32
(79.0, 84.0]      134
(84.0, 89.0]      228
(89.0, 94.0]      201
(94.0, 99.0]      153
(99.0, 104.0]      57
(104.0, 109.0]     18
(109.0, 114.0]      3
(114.0, 119.0]      1
Name: painoluokka, dtype: int64

--------------

          pelaaja joukkue syntymäkaupunki  maa  pituus  paino  ottelut  \
478   Olli Maatta     CHI       Jyväskylä  FIN  187.96   93.4       41   
788  Sami Vatanen     NJD       Jyväskylä  FIN  177.80   83.9       41   

     pisteet   painoluokka  
478       12  (89.0, 94.0]  
788       22  (79.0, 84.0]  


## 2. Arvosanat
Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/pisteet.txt on 1000 opiskelijan (kuvitteelliset) kurssisuoritukset. Lue tiedosto DataFrameen ja tee uusi sarake arvosana, joka määräytyy yhteispistemäärästä (teht1+teht2+teht3+labra1+labra2) seuraavan taulukon mukaan

Pisteitä vähintään | arvosana
--------------|--------------
    8|1
    12|2
    16|3
    19|4
    22|5
   
Mutta jos pistemäärä on vähemmän kuin 8 tai labrapistemäärä yhteensä (labra1+labra2) on vähemmän kuin 2, arvosanaksi tulee 0.

Tulosta sitten arvosanojen lukumäärät Serieksen value_counts()-metodilla.

pitäisi tulla määrät:

arvosana|lkm
----|---
0|312
1|169
2|199
3|157
4|133
5|30


In [5]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/pisteet.txt', sep=';', decimal=',')

df['yht'] = df['teht1']+df['teht2']+df['teht3']+df['labra1']+df['labra2']

# tai: 
# df['yht'] = df.sum(axis=1)

rajat = [0,8,12,16,19,22,df['yht'].max()+1]
arvosanat = [0,1,2,3,4,5]

df['arvosana'] = pd.cut(df['yht'],rajat, labels=arvosanat, right = False) # tässä laitetaan arvosana pelkillä pisteillä
df.loc[df['labra1']+df['labra2']<2, 'arvosana'] = 0 # tässä laitetaan huonoille labrapisteille arvosanat nolliksi

df['arvosana'].value_counts(sort=False)

0    312
1    169
2    199
3    157
4    133
5     30
Name: arvosana, dtype: int64

## 3. Juoksut
Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/juoksut.txt on (kuvitteellisia) juoksukilometrejä viikottain ja viikonpäivittäin taulukoituna.

Lue tiedosto DataFrameen ja:  
  * korvaa puuttuvat arvot nollalla
  * laske ja tulosta viikonpäivien (Mon, Tue, ...) mukaan keskiarvot (mean) juoksukilometreistä (ma pitäisi olla 9,58846...)
  * lisää DataFrameen sarake "yht", jossa on viikon kilometrit laskettu yhteen
  * lisää sarake "kerrat", jossa on kuinka monena päivänä ko. viikolla on kilometrejä > 0
  * lisää sarake "kova", jossa on True jos viikon kokonaiskilometrit on vähintään 75  TAI viikolla on vähintään kolme päivää joissa on vähintään 12 kilometriä
  * laske ja tulosta montako "kova"-viikkoa on ollut (pitäisi olla 17)

In [8]:
import pandas as pd
df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/juoksut.txt ', sep='|', index_col='week', decimal=',')

# korvataan puuttuvat arvot nollilla
df.fillna(0, inplace=True)

# keskiarvot sarakkeittain
print(df.mean())


# summat viikoittain, week on otettu indeksiksi, joten se ei tule mukaan. 
# muuten voisi tehdä esim. df['yht'] = df.iloc[:,1:].sum(axis=1)
# tai df['yht'] = df.apply(lambda x: x[1:].sum(), axis=1)
df['yht'] = df.sum(axis=1)

# tässä [:7] jotta yht-sarake ei tule mukaan
df['kerrat'] = df.apply(lambda x: x[:7][x>0].count(), axis=1)



def kova(x):
    if x['yht']>=75: return True
    if x[:7][x>=12].count()>=3: return True
    return False

df['kova'] = df.apply(kova, axis=1)

print(df.head(10))

print('\n----\n')
print('"kova"-viikkoja ',df['kova'].sum())

Mon    9.588462
Tue    8.178846
Wed    8.169231
Thu    8.311538
Fri    9.153846
Sat    8.253846
Sun    6.411538
dtype: float64
       Mon   Tue   Wed   Thu   Fri   Sat   Sun   yht  kerrat   kova
week                                                               
1     11.4   7.9  15.9  13.0  13.7  10.6   8.6  81.1       7   True
2      5.2   4.0   9.1   8.6   5.1   5.4   8.0  45.4       7  False
3      4.0  11.5  10.7   4.0  17.3   4.0  10.2  61.7       7  False
4     12.5   0.0   4.7  12.4   8.3  12.2   0.0  50.1       5   True
5     24.8   0.0   8.3   8.6   5.5  11.5   0.0  58.7       5  False
6     15.8   0.0   0.0   9.1  11.0   0.0   4.0  39.9       4  False
7      0.0  14.3  14.1   9.7   6.4  15.4   0.0  59.9       5   True
8      7.4   7.8   0.0  15.0   6.3   5.6   7.5  49.6       6  False
9     16.4   0.0  10.4   0.0   0.0   0.0  13.9  40.7       3  False
10     8.4   7.8   0.0  16.1   7.7   9.0  18.8  67.8       6  False

----

"kova"-viikkoja  17


## 4. NHL
Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/nhl.txt on NHL:n runkosarjan ottelutulokset kaudesta 2010-2011 kauteen 2018-2019.

outcome -sarakkeessa on merkintä REG, jos ottelu on päättynyt varsinaisella peliajalla, OT jos jatkoajalla ja SO jos ottelu on päättynyt vasta voittomaalikilpailussa.

Voittanut (enemmän maaleja tehnyt joukkue) saa 2 pistettä ja hävinnyt 0 pistettä, paitsi jos ottelu on ratkennut vasta jatkoajalla tai voittomaalikilpailussa, hävinnyt joukkue saa yhden pisteen.

Laske kuinka monta pistettä WPG (Winnipeg Jets)  on saanut tiedoston otteluista.  (pitäisi olla 696)  
Laske kuinka monta pistettä EDM (Edmonton Oilers) on saanut tiedoston otteluista. 


In [10]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/nhl.txt')

df['kotij'] = df['game'].str.split('-').str.get(0).str.strip()  # stripillä ylimääräiset välilyönnit pois alusta/lopusta
df['vierasj'] = df['game'].str.split('-').str.get(1).str.strip() 
df['kotim'] = df['goals'].str.split('-').str.get(0).astype(int) # luvuiksi jotta 11 > 2, merkkijonoina vertailtaessa 11 < 2
df['vierasm'] = df['goals'].str.split('-').str.get(1).astype(int)

df['kotipist'] = 0  # "alustetaan" pisteet nolliksi
df['vieraspist'] = 0

df.loc[df['kotim']>df['vierasm'] , 'kotipist'] = 2  # kotivoitosta 2 pistettä
df.loc[(df['outcome']!='REG') & (df['kotim']<df['vierasm']), 'kotipist'] = 1 # kotitappiosta 1 piste jos OT/SO

df.loc[df['kotim']<df['vierasm'] , 'vieraspist'] = 2 # vierasvoitosta 2 pistettä
df.loc[(df['outcome']!='REG') & (df['kotim']>df['vierasm']), 'vieraspist'] = 1 # vierastappiosta 1 piste jos OT/SO




print(df.sample(10))

print('\n--------------\n')

print('WPG ', df[df['kotij']=="WPG"]['kotipist'].sum()+df[df['vierasj']=="WPG"]['vieraspist'].sum())
print('EDM ', df[df['kotij']=="EDM"]['kotipist'].sum()+df[df['vierasj']=="EDM"]['vieraspist'].sum())

       season        date       game goals outcome kotij vierasj  kotim  \
3652  2013-14  13.12.2013  WPG - COL   3-4      SO   WPG     COL      3   
9358  2017-18    7.4.2018  TOR - MTL   4-2     REG   TOR     MTL      4   
9993  2018-19    4.1.2019  BUF - FLA   4-3     REG   BUF     FLA      4   
54    2010-11  16.10.2010  MTL - OTT   4-3     REG   MTL     OTT      4   
8547  2017-18  10.12.2017  TBL - WPG   4-3      OT   TBL     WPG      4   
3239  2013-14  11.10.2013  VAN - SJS   1-4     REG   VAN     SJS      1   
2593  2012-13    6.2.2013  STL - NSH   1-6     REG   STL     NSH      1   
1305  2011-12  19.10.2011  VAN - NYR   0-4     REG   VAN     NYR      0   
4005  2013-14    1.2.2014  BOS - EDM   4-0     REG   BOS     EDM      4   
8106  2017-18   5.10.2017  BUF - MTL   2-3      SO   BUF     MTL      2   

      vierasm  kotipist  vieraspist  
3652        4         1           2  
9358        2         2           0  
9993        3         2           0  
54          3         

## 5. Tulokset
Tiedostossa http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/tulokset.txt on erään suunnistuskisan tuloksia, kenttäerottimena sarkain.
Kilpailija-kentässä on data muodossa **M10 Aaltio Aapo (KuoSu)** missä  
  * M = sukupuoli (M tai N)  
  * 10 = matka (10, 15 tai 25)  
  * Aaltio = sukunimi  
  * Aapo = etunimi  
  * KuoSu = seura. Seura voi myös puuttua, jolloin kentässä ei ole myöskään sulkuja  

Nimi voi olla myös 3-osainen (esim. **M15 af Hällström Fabian (OK77)**). Näissä kaksi ensimmäistä osaa kuuluvat sukunimeen ja kolmas etunimeen.

Lue tiedot DataFrameen, ja pura kilpailija-kentästä eri sarakkeisiin etunimi, sukunimi, seura, matka ja sukupuoli.

Tulosta sitten naisten 10 km nopein juoksija.  
(ajat ovat tässä siinä muodossa, että niitä voidaan lajitella sellaisinaan merkkijonoina).

In [11]:
import pandas as pd

df = pd.read_csv('http://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta2/tulokset.txt', sep='\t')


# 1. merkki sukupuoleksi
df['sukupuoli'] = df['kilpailija'].str[0]  
# toinen ja kolmas merkki matkaksi
df['matka'] = df['kilpailija'].str[1:3].astype(int)

# sulkua edeltävän osan viimeinen välilyönnin jälkeinen merkkijono etunimeksi
df['etunimi'] = df['kilpailija'].str.split('(').str.get(0).str.strip().str.split(' ').str.get(-1) 

# sulun jälkeinen pätkä ilman viimeistä )-merkkiä seuraksi. Jos sitä ei ole, tulee NaN
df['seura'] = df['kilpailija'].str.split('(').str.get(1).str[0:-1]

# tässä "apujuttuna" koko nimi, n=1 kertoo että ainoastaan ensimmäisestä välilyönnistä splitataan
df['kokonimi'] = df['kilpailija'].str.split('(').str.get(0).str.strip().str.split(' ', n=1).str.get(1)

# sukunimi rsplitillä, joka n=1:llä katkaisee vain viimeisestä välilyönnistä 
df['sukunimi'] = df['kokonimi'].str.rsplit(' ', n=1).str.get(0)

# tapa2, [::-1] kääntää merkkijonon toisin päin
#df['sukunimi'] = df['kokonimi'].str[::-1].str.split(' ', n=1).str.get(1).str[::-1]

# pudotetaan turhat sarakkeet pois
df.drop(['kilpailija', 'kokonimi'], axis=1, inplace = True)


print(df.head())

print('\n-----------\n')

# nopein 10 km nainen
print(df[(df['matka']==10) & (df['sukupuoli']=='N')].sort_values('loppuaika').head(1))

   kilpnro loppuaika sukupuoli  matka    etunimi  seura      sukunimi
0        3   1:24:11         M     10       Aapo  KuoSu        Aaltio
1        6   2:45:02         M     15     Tuomas   EsAk         Aalto
2       13   1:50:48         M     15     Fabian   OK77  af Hällström
3        7   2:06:39         N     10  Pia-Maria    NaN       Ahlfors
4        4   4:09:09         N     15  Kristiina   Raus      Ahlqvist

-----------

     kilpnro loppuaika sukupuoli  matka etunimi   seura sukunimi
516      591   1:21:40         N     10  Ulrika  Koovee   Uotila
